In [1]:
import numpy as np
import matplotlib.pyplot as plt
import torch
from torch import nn
from torch import optim
import torch.nn.functional as F
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader
import time
import pandas as pd

from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

In [2]:
pip install torch

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'D:\Anaconda3\python.exe -m pip install --upgrade pip' command.


In [2]:
frameNum = 60 

# load data
dfs = []
for i in range(1,33):
  for j in range(1,41):
    filename = f"./Files/User{i}_Video{j}.dat"
    cols = np.arange(frameNum)
    df = pd.read_csv(filename, header = None, usecols = cols, delimiter=' ')   
    dfs.append(df.values)
    #print('participant%dvideo%d.txt'%(i,j))

dfs = np.array(dfs)
print('dataLoaded:')
print(dfs.shape)

dataLoaded:
(1280, 1024, 60)


In [4]:
df

,0,1,2,3,4,5,6,7,8,9,...,50,51,52,53,54,55,56,57,58,59
0,5.744679e-05,2.553555e-04,6.081591e-04,0.001022,1.358385e-03,1.512967e-03,1.435924e-03,0.001143,6.851798e-04,2.540377e-04,...,1.440838e-04,2.968252e-04,3.327871e-04,1.907146e-04,5.777137e-05,3.531563e-07,1.131937e-04,3.096933e-04,4.225250e-04,4.598667e-04
1,1.896244e-04,5.140935e-04,9.558036e-04,0.001427,1.749479e-03,1.818524e-03,1.688201e-03,0.001282,7.126611e-04,2.659181e-04,...,3.900513e-05,6.972865e-05,8.060219e-05,5.384183e-05,1.027467e-05,8.422783e-06,8.241706e-05,1.739351e-04,2.199536e-04,2.423155e-04
2,4.424911e-04,8.062397e-04,1.327554e-03,0.001792,1.963859e-03,1.990143e-03,1.746486e-03,0.001241,7.093119e-04,2.072976e-04,...,9.325482e-06,8.649716e-06,1.195799e-05,2.304283e-06,5.011289e-06,3.122549e-05,7.142586e-05,1.206939e-04,1.962808e-04,2.646049e-04
3,6.990651e-04,1.194994e-03,1.604066e-03,0.001909,2.080131e-03,1.977483e-03,1.597441e-03,0.001060,5.215706e-04,1.201247e-04,...,1.236240e-05,3.730674e-06,4.530567e-07,8.414927e-06,2.729891e-05,6.257979e-05,1.118084e-04,1.828313e-04,2.560754e-04,2.950363e-04
4,9.715428e-04,1.380071e-03,1.751842e-03,0.001996,1.949584e-03,1.673408e-03,1.313303e-03,0.000843,3.323800e-04,3.403212e-05,...,1.602134e-05,2.763704e-06,2.016952e-06,3.032129e-05,6.418142e-05,1.023652e-04,1.876355e-04,2.840458e-04,3.488021e-04,4.249631e-04
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1019,3.213220e-06,1.728500e-06,1.079931e-06,0.000001,7.633227e-07,1.424721e-06,2.315452e-06,0.000001,1.238352e-06,1.640884e-06,...,7.683036e-07,9.850071e-07,3.738347e-07,1.770216e-07,2.586192e-07,2.687064e-07,4.844050e-07,9.182721e-07,2.673959e-07,7.516916e-09
1020,2.324575e-07,1.569091e-06,3.535182e-06,0.000002,6.087443e-07,4.513868e-07,2.613192e-06,0.000004,6.983612e-07,1.214008e-09,...,3.827605e-08,1.169237e-07,1.233719e-07,4.624929e-08,1.528097e-07,6.242575e-07,2.100241e-07,3.001984e-08,6.512785e-08,5.664144e-07
1021,3.131241e-06,1.464914e-06,2.107083e-07,0.000001,1.875283e-06,6.668432e-07,1.704003e-06,0.000002,4.193310e-07,8.427626e-07,...,1.094986e-07,1.536073e-08,3.565909e-07,3.117275e-08,2.581929e-07,1.334709e-08,1.250931e-08,7.376728e-09,1.594658e-07,8.691596e-08
1022,1.391200e-06,2.521965e-07,9.026174e-07,0.000003,1.429012e-06,2.175078e-07,7.982079e-07,0.000004,2.299245e-06,9.996390e-09,...,1.519980e-09,5.100703e-08,9.030283e-07,3.390141e-07,3.083695e-07,1.549144e-07,2.946007e-07,3.212838e-07,4.275782e-09,8.383055e-08


In [3]:
# normalize
x_min = dfs.min(axis = (1,2),keepdims=True)
x_max = dfs.max(axis = (1,2),keepdims=True)
dfs_normal = (dfs-x_min)/(x_max-x_min)
print(dfs_normal.shape)

(1280, 1024, 60)


In [4]:
depth = 3
# divide frames ,or 60s is too long for a single 3dinput
reshape_dfs = np.split(dfs_normal, frameNum/depth, axis=2)
reshape_dfs = np.array(reshape_dfs)
reshape_dfs = np.reshape(reshape_dfs,[-1,1024,depth])
print(reshape_dfs.shape)

(25600, 1024, 3)


In [5]:
# load label
label_dfv = pd.read_csv('./labels_class_0.csv',
    names = ['valence'], header=None, delimiter=',' )

label_dfa = pd.read_csv('./labels_class_1.csv',
    names = ['arousal'], header=None, delimiter=',' )

label_dfd = pd.read_csv('./labels_class_2.csv',
    names = ['dominance'], header=None, delimiter=',' )

label_dfl = pd.read_csv('./labels_class_3.csv',
    names = ['liking'], header=None, delimiter=',' )

#label_dfv
print(label_dfv.shape, label_dfa.shape, label_dfd.shape, label_dfl.shape)

(1280, 1) (1280, 1) (1280, 1) (1280, 1)


In [21]:
label_dfv.iloc[5,0]

1

In [6]:
label_dfva = pd.DataFrame(columns=['valencearousal'])
lb = {'00':0, '01':1, '10':2, '11':3}

In [7]:
for i in range(1280):
    label_dfva = label_dfva.append({'valencearousal':lb[str(label_dfv.iloc[i,0]) + str(label_dfa.iloc[i,0])]},ignore_index = True)
label_dfva

,valencearousal
0,3
1,3
2,3
3,3
4,2
...,...
1275,1
1276,1
1277,1
1278,1


In [29]:
label_dfva['valencearousal'].value_counts()

3    531
1    285
2    277
0    187
Name: valencearousal, dtype: int64

In [17]:
labelv = np.tile(label_dfv['valence'].astype(int).values,20)
labela = np.tile(label_dfa['arousal'].astype(int).values,20)
labeld = np.tile(label_dfd['dominance'].astype(int).values,20)
labell = np.tile(label_dfl['liking'].astype(int).values,20)
print(labelv.shape, labela.shape, labeld.shape, labell.shape)

(25600,) (25600,) (25600,) (25600,)


In [8]:
labelva = np.tile(label_dfva['valencearousal'].astype(int).values,20)
print(labelva.shape)

(25600,)


In [9]:
# divive train & test
x_train, x_test, y_train, y_test = train_test_split(reshape_dfs, labelva, test_size=0.2)

x_train = torch.from_numpy(x_train).float()
y_train = torch.from_numpy(y_train).long()
x_test = torch.from_numpy(x_test).float()
y_test = torch.from_numpy(y_test).long()

print(x_train.shape,  y_train.shape,x_test.shape, y_test.shape)

torch.Size([20480, 1024, 3]) torch.Size([20480]) torch.Size([5120, 1024, 3]) torch.Size([5120])


In [10]:
class cnn_classifier(nn.Module):
  def __init__(self):
    super().__init__()
    self.conv11 = nn.Conv3d(in_channels=1, out_channels=32, kernel_size=3, stride=1, padding=1)
    self.conv12 = nn.Conv3d(in_channels=32, out_channels=32, kernel_size=3, stride=1, padding=1)
    self.pool1 = nn.MaxPool3d(kernel_size=2, padding=(0,0,1))
    
    self.conv21 = nn.Conv3d(in_channels=32, out_channels=64, kernel_size=3, stride=1, padding=1)
    self.conv22 = nn.Conv3d(in_channels=64, out_channels=64, kernel_size=3, stride=1, padding=1)
    self.pool2 = nn.MaxPool3d(kernel_size=2, padding=(0,0,1))
    
    self.conv31 = nn.Conv3d(in_channels=64, out_channels=128, kernel_size=3, stride=1, padding=1)
    self.conv32 = nn.Conv3d(in_channels=128, out_channels=128, kernel_size=3, stride=1, padding=1)
    self.pool3 = nn.MaxPool3d(kernel_size=2, padding=0)
    

    self.fc_layer = nn.Linear(128*4*4*1, 2)
    
    self.dropout_layer = nn.Dropout(p=0.5)

  def forward(self, xb):
    h1 = self.conv11(xb)
    h1 = self.conv12(h1)
    h1 = self.dropout_layer(h1)
    h1 = self.pool1(h1)
    h1 = F.relu(h1)

    h2 = self.conv21(h1)
    h2 = self.conv22(h2)
    #h2 = self.dropout_layer(h2)
    h2 = self.pool2(h2)
    h2 = F.relu(h2) 

    h3 = self.conv31(h2)
    h3 = self.conv32(h3)
    #h3 = self.dropout_layer(h3)
    h3 = self.pool3(h3)
    h3 = F.relu(h3) 
    
    
    # flatten the output from conv layers before feeind it to FC layer
    flatten = h3.view(-1, 128*4*4*1)
    out = self.fc_layer(flatten)
    #out = self.dropout_layer(out)
    return out

In [11]:
def train_model(model, x_train, y_train, x_test, y_test, epochs=35 , batch_size=32, lr=0.0001, weight_decay=0):
    # data
    train_dataset = TensorDataset(x_train, y_train)
    train_data_loader = DataLoader(train_dataset, batch_size=batch_size)

    # loss function
    loss_func = F.cross_entropy

    # optimizer
    #optimizer = optim.SGD(model.parameters(), lr=lr, weight_decay=weight_decay)
    optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)

    # figure
    train_a = list([])
    test_a = list([])

    # training loop
    for epoch in range(epochs):
        model.train()
        tic = time.time()
        acc_train = []
        for xb, yb in train_data_loader:    
            xb, yb = xb.to(device), yb.to(device)
            pred = model(xb)
      
        optimizer.zero_grad()
        optimizer.step()
        acc_train.append(pred.detach().argmax(1).eq(yb).float().mean().cpu().numpy())
        acc_train = np.mean(acc_train)
        toc = time.time()
    
        with torch.no_grad():
            model.eval()
            y_pred = model(x_test.to(device))
            acc = y_pred.argmax(1).eq(y_test.to(device)).float().mean().cpu().numpy()

        train_a.append(acc_train)
        test_a.append(acc)
        print('Epoch %d : train_acc: %f, test_acc: %f, running time: %d'% (epoch, acc_train, acc, toc-tic))

    # print accuracies
    print('Maximum Training Accuracy: ',max(train_a))
    print('Maximum Testing Accuracy: ',max(test_a))
    
    # draw an accuracy figure
    plt.plot(train_a,'y.-.')
    plt.plot(test_a,'.-.')
    plt.xlabel('epoch')
    plt.ylabel('accuracy')

In [ ]:
model = cnn_classifier()
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)
model = model.to(device)
train_model(model, x_train.view(-1, 1, 32, 32, 3), y_train, x_test.view(-1, 1, 32, 32, 3), y_test)

cpu


In [14]:
# divide train & test
x_train, x_test, y_train, y_test = train_test_split(reshape_dfs, labela, test_size=0.2)

x_train = torch.from_numpy(x_train).float()
y_train = torch.from_numpy(y_train).long()
x_test = torch.from_numpy(x_test).float()
y_test = torch.from_numpy(y_test).long()

print(x_train.shape,  y_train.shape,x_test.shape, y_test.shape)

torch.Size([20480, 1024, 3]) torch.Size([20480]) torch.Size([5120, 1024, 3]) torch.Size([5120])


In [15]:
train_model(model, x_train.view(-1, 1, 32, 32, 3), y_train, x_test.view(-1, 1, 32, 32, 3), y_test)

Loss at epoch 0 : 0.681590, train_acc: 0.750000, test_acc: 0.642969, running time: 64
Loss at epoch 1 : 0.674492, train_acc: 0.750000, test_acc: 0.642969, running time: 66
Loss at epoch 2 : 0.666143, train_acc: 0.750000, test_acc: 0.642969, running time: 66
Loss at epoch 3 : 0.656142, train_acc: 0.750000, test_acc: 0.642969, running time: 66
Loss at epoch 4 : 0.643261, train_acc: 0.750000, test_acc: 0.642969, running time: 67
Loss at epoch 5 : 0.626970, train_acc: 0.750000, test_acc: 0.642969, running time: 66
Loss at epoch 6 : 0.607250, train_acc: 0.750000, test_acc: 0.642969, running time: 68


KeyboardInterrupt: 

In [2]:
torch.cuda.is_available()

False

In [48]:
if torch.cuda.is_available():  
  dev = "cuda:0" 
else:  
  dev = "cpu"  
device = torch.device(dev)  
a = torch.zeros(4,3)    
a = a.to(device)

In [49]:
device

device(type='cpu')